In [1]:
# pylint: disable=missing-module-docstring
%load_ext autoreload
%autoreload 1
%aimport simulation

from collections import Counter
import os
import pandas as pd
import plotly.express as px

from simulation.parameters import Param
from simulation.model import Model

In [2]:
# Path to the outputs folder
OUTPUT_DIR = "../outputs/"

## Running the model

In [3]:
# Run the model
model = Model(param=Param(), run_number=0)
model.run()

In [4]:
# Preview some of the results
model.audit_list[0:5]

[{'time': 1095, 'asu_occupancy': 7, 'rehab_occupancy': 12},
 {'time': 1096, 'asu_occupancy': 5, 'rehab_occupancy': 11},
 {'time': 1097, 'asu_occupancy': 5, 'rehab_occupancy': 10},
 {'time': 1098, 'asu_occupancy': 4, 'rehab_occupancy': 10},
 {'time': 1099, 'asu_occupancy': 5, 'rehab_occupancy': 10}]

## Figure 1

**Figure 1.** Simulation probability density function for occupancy of an acute stroke unit.

**Relevant functions...**

In [5]:
def get_occupancy_freq(unit):
    """
    Count the frequency each occupancy level was observed at within the
    interval audit for the specified unit.

    Parameters
    ----------
    unit: str
        Name of unit to plot ("asu", "rehab").

    Returns
    -------
    df: pd.DataFrame
        Dataframe with two columns: the number of beds ("beds") and the
        frequency this was observed in the audit ("freq").
    """
    # Get occupancy audit results
    occ_audit = pd.DataFrame(model.audit_list)

    # Get the frequency of each number of beds in the specified unit
    frequency = Counter(occ_audit[f"{unit}_occupancy"])

    # Fill in any gaps - find the maximum bed count, then create complete
    # dictionary include bed counts which were never observed
    min_bed_count = min(frequency.keys())
    max_bed_count = max(frequency.keys())
    complete_frequency = {i: frequency.get(i, 0)
                          for i in range(min_bed_count, max_bed_count + 1)}

    # Convert into a dataframe
    df = pd.DataFrame(
        complete_frequency.items(), columns=["beds", "freq"])

    # Add column with frequencies converted to proportions
    df["pct"] = df["freq"] / df["freq"].sum()

    # Add column with the cumulative percentage
    df["c_pct"] = df["pct"].cumsum()

    return df

In [6]:
def plot_occupancy_freq(df, unit, file, path=OUTPUT_DIR):
    """
    Plot the frequency at which each occupancy level was observed in the audit.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe output by `get_occupancy_freq()` containing the frequency
        each occupancy was observed at.
    unit: str
        Name of unit ("asu", "rehab")
    file: str
        Filename to save figure to (e.g. "figure.png").
    path: str
        Path to save file to (excluding filename).
    """
    # Create plot
    fig = px.bar(df, x="beds", y="pct", color_discrete_sequence=["black"])

    # Specify axis labels, theme and dimensions
    if unit == "asu":
        unit_lab = "acute"
    elif unit == "rehab":
        unit_lab = "rehabilitation"
    fig.update_layout(
        xaxis_title=f"No. patients in {unit_lab} unit",
        yaxis_title="% observations",
        template="plotly_white",
        height=450,
        width=800
    )

    # Add box around figure, and set tick spacing to 1
    fig.update_xaxes(linecolor="black", mirror=True, dtick=1)
    fig.update_yaxes(linecolor="black", mirror=True, tickformat=",.0%")

    # Show figure
    fig.show()

    # Save figure
    fig.write_image(os.path.join(path, file))

**Generate plots...**

(the article just includes a plot for the acute stroke unit).

In [7]:
# Acute stroke unit
asu_occupancy = get_occupancy_freq("asu")
plot_occupancy_freq(asu_occupancy, unit="asu",
                    file="occupancy_freq_asu.png")

# Rehabilitation unit
rehab_occupancy = get_occupancy_freq("rehab")
plot_occupancy_freq(rehab_occupancy, unit="rehab",
                    file="occupancy_freq_rehab.png")

## Figure 3

**Figure 3**. Simulated trade-off between the probability that a patient is delayed and the no. of acute beds available.

### Explanation of the method for calculating blocking probability

We can use our frequency and cumulative frequency of occupied beds from the simulation to calculate blocking probability.

Above, we created the tables...

In [8]:
asu_occupancy.head()

,beds,freq,pct,c_pct
0,1,1,0.000548,0.000548
1,2,12,0.006575,0.007123
2,3,39,0.021370,0.028493
3,4,76,0.041644,0.070137
4,5,134,0.073425,0.143562


We can interpret...

* `pct` as the probability of having exactly x beds occupied.
* `c_pct` as the probability of having x or fewer beds occupied.

We can then calculate `pct/c_pct`, which is the probability of delay when the system has exactly x beds occupied.

For example:

| Beds | `pct` | `c_pct` | Probability of delay |
| - | - | - | - |
| 7 | 0.2 | 0.2 | 1.0 |
| 8 | 0.3 | 0.5 | 0.6 |
| 9 | 0.4 | 0.9 | 0.44 |
| 10 | 0.1 | 1.0 | 0.1 |

Interpretation for 8 beds:

* If we **randomly select a day when the occupancy is 8 or fewer beds**, there's a **60%** chance that the occupancy will be **exactly 8 beds (rather than 7 beds)**.

This can then be connected to the probability of delay by thinking about system capacity:

* If we assume that the unit has a total of 8 beds, then when 8 beds are occupied, the unit is at **full capacity**.
* Any new patients arriving when 8 beds are occupied would experience a delay.
* So 0.6 represents the probability that, given we're at or below capacity (7 or 8 beds), we're actually at full capacity (8 beds)

In other words, `pct/c_pct` is the probability that a new arrival will experience a delay when the system has exactly x beds occupied, given that the capacity of the system is x beds.

### Implementing this method

In [9]:
def plot_delay_prob(df, unit, file, path=OUTPUT_DIR):
    """
    Plot the simulated trade-off between the probability of delay and the
    number of beds available.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe output by `get_occupancy_freq()` containing the frequency
        each occupancy was observed at.
    unit: str
        Name of unit ("asu", "rehab")
    file: str
        Filename to save figure to (e.g. "figure.png").
    path: str
        Path to save file to (excluding filename).
    """
    # Calculate the probability of delay
    df["prob_delay"] = df["pct"] / df["c_pct"]

    # Create the step plot
    fig = px.line(df, x="beds", y="prob_delay",
                  color_discrete_sequence=["black"])
    fig.update_traces(mode="lines", line_shape="hv")

    # Add axis labels, set theme and dimensions
    if unit == "asu":
        unit_lab = "acute"
    elif unit == "rehab":
        unit_lab = "rehabilitation"
    fig.update_layout(
        xaxis_title=f"No. of {unit_lab} beds available",
        yaxis_title="Probability of delay",
        template="simple_white",
        height=450,
        width=800
    )

    # Set tick frequency and adjust axis
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(dtick=0.1, range=[0, 1])

    # Show figure
    fig.show()

    # Save figure
    fig.write_image(os.path.join(path, file))

In [10]:
plot_delay_prob(asu_occupancy, unit="asu", file="delay_prob_asu.png")
plot_delay_prob(rehab_occupancy, unit="rehab", file="delay_prob_rehab.png")